In [1]:
import dask.dataframe as ddf
import pandas as pd
import numpy as np

In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:51285 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.50 GB


In [3]:
users_df = pd.read_csv('data/hash.txt', sep=" ", header=None, nrows=1000000)
users_df.columns = ["User_ID"]
users_df['Binary'] = users_df.User_ID

In [4]:
users_df.head(10)

,User_ID,Binary
0,844082e02a27ddee8d99ea1af94a2969,844082e02a27ddee8d99ea1af94a2969
1,ff96d6665b5c59d3a70bb8f2ba4f10be,ff96d6665b5c59d3a70bb8f2ba4f10be
2,b64a85884e2b159829331c19e05dbac9,b64a85884e2b159829331c19e05dbac9
3,1c8836719e84867c26ba2cfeb372c53d,1c8836719e84867c26ba2cfeb372c53d
4,b66f73ffd9008d9c99159e164261df51,b66f73ffd9008d9c99159e164261df51
5,5f414364fde0ca5a72b5a4313d5266aa,5f414364fde0ca5a72b5a4313d5266aa
6,fdcb0c09722ea060d81cba08681a29b9,fdcb0c09722ea060d81cba08681a29b9
7,0ebca291ec5fa3f2146eb1c3a219fe13,0ebca291ec5fa3f2146eb1c3a219fe13
8,1ed24bee346c4c34a491a3db6ce41e73,1ed24bee346c4c34a491a3db6ce41e73
9,4d6289943c24816060240ddff3c530d7,4d6289943c24816060240ddff3c530d7


### Convert User_IDs into binary

We will be using a hash function table for this such as the following:

##### Rational

He feel like we need to justify why we have come up with the following has table:

We started looking into different hash functions, all of them resulting in different issues:

- Convert string into binary: We used the function ord() in python, but it was giving very bas results
- Convert letters into integers using ascii and convert each individual integer into its binary: We found this also to be problematic since most strings (except the number 0) would start with 1's. It also gave a very un-evenly distributed probabilities between 0 and 1.

We then finally decided to generate a binary value of 5 digits (32 possible combinations) and then also add the 4 remaining values using smaller binary values in order to keep the distribution uniform

In [73]:
asciiDict = {chr(i): i for i in range(97, 123)}
list_values = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] + list(asciiDict.keys())
list_binary = ['0', '1', '11', '00'] + list(get_bucket_groups(32).keys())

df_hash = pd.DataFrame(columns=['Original', 'Binary'])
df_hash['Original'] = list_values
df_hash['Binary'] = list_binary
df_hash['# 0'] = [i.count('0') for i in list_binary]
df_hash['# 1'] = [i.count('1') for i in list_binary]

In [74]:
df_hash

,Original,Binary,# 0,# 1
0,0,0,1,0
1,1,1,0,1
2,2,11,0,2
3,3,00,2,0
4,4,00000,5,0
5,5,00001,4,1
6,6,00010,4,1
7,7,00011,3,2
8,8,00100,4,1
9,9,00101,3,2


In [75]:
hash_dictionary = dict(zip(df_hash.Original, df_hash.Binary))

In [76]:
hash_dictionary

{'0': '0',
 '1': '1',
 '2': '11',
 '3': '00',
 '4': '00000',
 '5': '00001',
 '6': '00010',
 '7': '00011',
 '8': '00100',
 '9': '00101',
 'a': '00110',
 'b': '00111',
 'c': '01000',
 'd': '01001',
 'e': '01010',
 'f': '01011',
 'g': '01100',
 'h': '01101',
 'i': '01110',
 'j': '01111',
 'k': '10000',
 'l': '10001',
 'm': '10010',
 'n': '10011',
 'o': '10100',
 'p': '10101',
 'q': '10110',
 'r': '10111',
 's': '11000',
 't': '11001',
 'u': '11010',
 'v': '11011',
 'w': '11100',
 'x': '11101',
 'y': '11110',
 'z': '11111'}

In [77]:
#asciiDict = {chr(i): i for i in range(97, 123)}

#def string_to_integers(string):
    
#    list_string = list(string)
#    i = -1
#    for letter in list_string:
#        i+=1
#        if letter.isdigit():
#            continue
#        else:
#            list_string[i] = str(asciiDict[letter])
#    integers = ''.join(list_string)
#    return integers

#def decimal_to_binary(integer, decimals=None):
#    binary = ''
#    i = integer
#    while i >= 1:
#        binary = str(int(i%2)) + binary
#        i = i/2
#    if decimals:
#        binary = '0'*(decimals-len(binary)) + binary
#    return binary if binary else '0'

#def string_to_binary(string):
#   
#    string_as_integers = string_to_integers(string)
#    
#    binary = ''
#    for integer in string_as_integers:
#        binary = binary + decimal_to_binary(int(integer))
#    return binary

def string_to_binary(string):
    
    binary = ''
    for letter in string:
        binary = binary + hash_dictionary[letter]
    return binary

In [78]:
def string_to_binary_df(df):
    
    df['Binary'] = df.User_ID.apply(lambda x: string_to_binary(x))
    
    return df

In [79]:
dask_dataframe = ddf.from_pandas(users_df, npartitions=20)

In [80]:
%%time
users_df_binary = dask_dataframe.map_partitions(string_to_binary_df, meta=users_df).compute()

Wall time: 11.7 s


In [81]:
users_df_binary.head()

,User_ID,Binary
0,844082e02a27ddee8d99ea1af94a2969,0010000000000000001001101010011001101100011010...
1,ff96d6665b5c59d3a70bb8f2ba4f10be,0101101011001010001001001000100001000010000010...
2,b64a85884e2b159829331c19e05dbac9,0011100010000000011000100000010010000100000000...
3,1c8836719e84867c26ba2cfeb372c53d,1010000010000100000001000011100101010100010000...
4,b66f73ffd9008d9c99159e164261df51,0011100010000100101100011000101101011010010010...


### Split binary values into categories (m) based on a new hash function

In [94]:
def split_binary(df, m=64, column='Binary'):
    """
    This function will take a df with binary values (binary values are the hashed unique user ids) 
    and splits them into a set of buckets (m). For each of these buckets the function will also compute 
    the maximum number of zero's at the end of each binary number. The function will then return this 
    list which will then be used to estimate the distinct count value
    
    df: Dataframe with a column of binary values
    m: Number of buckets
    """
    
    len_buckets = int(np.log2(m))
    max_zeros_per_bucket = get_bucket_groups(m)

    for index, user_id in df[[column]].iterrows():
        group = user_id[0][:len_buckets]
        num_final_zeros = len(user_id[0]) - len(user_id[0].rstrip('0'))
        max_zeros_per_bucket[group] = max(max_zeros_per_bucket[group], num_final_zeros)
    
    return list(max_zeros_per_bucket.values())

In [97]:
def get_bucket_groups(m=64):
    """
    Function that will give us the initial binary values to bucket our hashed binary values
    """
    buckets = {}
    len_binary = int(np.log2(m))
    for i in range(m):
        bucket_binary = decimal_to_binary(i, decimals=len_binary)
        buckets[bucket_binary] = 0
    return buckets

In [98]:
def compute_alpha(m):
    """
    Function to compute the alpha value used correct a systematic multiplicative bias
    Instead of computing the integral we have taken the constant values from wikipedia
    """
    
    if m == 16:
        alpha = 0.673
    elif m == 32:
        alpha = 0.697
    elif m == 64:
        alpha = 0.709
    else:
        alpha = 0.7213 / (1+ (1.079/m))
    
    return alpha
    

In [99]:
%%time
max_per_bucket = split_binary(users_df_binary)

Wall time: 4min 25s


In [100]:
def hyperloglog_estimate(max_zeros):
    """
    Returns the estimated hyperloglog estimate and its estimated error
    
    max_zeros: A list with all the max values over which the harmonic mean will be applied
    """
    
    max_zeros = np.array(max_zeros)
    m = len(max_zeros)
    
    Z = float(2)**(-max_zeros)
    Z = 1 / (np.sum(Z))
    
    estimate = compute_alpha(m) * m**2 * Z
    
    error = 1.04 / np.sqrt(m)
    
    return estimate, error

In [103]:
hyperloglog_estimate(max_per_bucket)

(45128.226769549074, 0.13)